In [1]:
import nltk
import pandas as pd
import numpy as np
import csv
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import metrics
%matplotlib inline
from google.colab import files
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, cross_val_score


In [2]:
uploaded = files.upload()
data = pd.read_csv('dataset01.txt', sep = '\n')
print(data.head())

Saving dataset01.txt to dataset01.txt
                                            all_data
0                                     >iTELL-Chr1_61
1  CACACCACACCCACACACCCACACACCACACCACACACCACACCAC...
2                                                  0
3                                   >iYAL067W-A_5085
4  AATTTTTATAGGTCGACCCTTCTGTCGCTTACTGGGTTGATTATCT...


In [3]:
all=data["all_data"].tolist()
cls=[]
sequence=[]
label=[]

In [4]:
length = len(all)
for i in range(length):
   if (i%3)==0:
             cls.append(all[i])
   else:
      if(all[i]== "0" or all[i]=="1"):
        label.append(all[i])
      else:
         sequence.append(all[i])

print(len(cls))
print(len(label))
print(len(sequence))

14965
14965
14965


In [5]:
df = pd.DataFrame(list(zip(cls, label, sequence)),
               columns =['class' , 'label', 'sequence'])
df.head(10)

,class,label,sequence
0,>iTELL-Chr1_61,0,CACACCACACCCACACACCCACACACCACACCACACACCACACCAC...
1,>iYAL067W-A_5085,0,AATTTTTATAGGTCGACCCTTCTGTCGCTTACTGGGTTGATTATCT...
2,>iYAL067W-A_6786,0,AATTATATTTCCATCAGCTCAATACCGCAGTACTTTGAAACCTGAT...
3,>iYAL067W-A_7060,0,AACAATAGTGGGTATGAGTAAAGATATATAGATCGATATTTTGAAT...
4,>YAL067C_SEO1_8061,1,CAAAGATTTCAACCATAGTAGGTATGCCCCAGATGAAACATTACTG...
5,>YAL067C_SEO1_8340,1,CACAATAGCGTCAATAATAAAGTTCCATCTCCATCCCTCTAAACCA...
6,>iYAL067C_9509,1,GTTTTTTTCCAGTGACACAATCTTTACCATTACACAGTTTTTACTA...
7,>iYAL067C_10442,0,CAGAAACATAAAATTATATCACTTTATTTCATATGGTTTCATGCTT...
8,>YAL065C_YAL065C_11570,1,AGATACCTCGAAACAAACTCTATGTAAACACTTATTTTATTGTGGT...
9,>iYAL064W-B_13286,1,ATGACTGGAGAAGAAGTGACTAGAGAAGAAGTGACTGAGGAAGAAA...


In [6]:
X = df['sequence']
Y = df['label']

In [7]:
def getKmers(sequence, size=5):
    return [sequence[x:x+size].lower() for x in range(len(sequence) - size + 1)]
    

In [8]:
df['words'] = df.apply(lambda x: getKmers(x['sequence']), axis=1)
df = df.drop('sequence', axis=1)
df.head(10)             

,class,label,words
0,>iTELL-Chr1_61,0,"[cacac, acacc, cacca, accac, ccaca, cacac, aca..."
1,>iYAL067W-A_5085,0,"[aattt, atttt, ttttt, tttta, tttat, ttata, tat..."
2,>iYAL067W-A_6786,0,"[aatta, attat, ttata, tatat, atatt, tattt, att..."
3,>iYAL067W-A_7060,0,"[aacaa, acaat, caata, aatag, atagt, tagtg, agt..."
4,>YAL067C_SEO1_8061,1,"[caaag, aaaga, aagat, agatt, gattt, atttc, ttt..."
5,>YAL067C_SEO1_8340,1,"[cacaa, acaat, caata, aatag, atagc, tagcg, agc..."
6,>iYAL067C_9509,1,"[gtttt, ttttt, ttttt, ttttt, ttttc, tttcc, ttc..."
7,>iYAL067C_10442,0,"[cagaa, agaaa, gaaac, aaaca, aacat, acata, cat..."
8,>YAL065C_YAL065C_11570,1,"[agata, gatac, atacc, tacct, acctc, cctcg, ctc..."
9,>iYAL064W-B_13286,1,"[atgac, tgact, gactg, actgg, ctgga, tggag, gga..."


In [9]:
dna_texts = list(df['words'])
for item in range(len(dna_texts)):
    dna_texts[item] = ' '.join(dna_texts[item])
y_data = df['label'].values

Bag of words

In [10]:
cv = CountVectorizer(ngram_range=(4,4))
X = cv.fit_transform(dna_texts)
print(X.shape)

(14965, 65536)


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y_data, test_size = 0.25,  shuffle = True) 

print(X_train.shape)
print(X_test.shape)                                         

(11223, 65536)
(3742, 65536)


In [12]:
scoring = 'accuracy'

names = ['K Nearest Neighbors', 'Decision Tree', 'Random Forest', 'AddaBoost', 'Multinomial NB',
         'SVM Linear', 'Logistic Regression']
Classifiers = [
    KNeighborsClassifier(n_neighbors = 3),
    DecisionTreeClassifier(),
    RandomForestClassifier(n_estimators = 100, max_features = 1 ),
    AdaBoostClassifier(),
    MultinomialNB(alpha=0.1),
    svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto'),
    LogisticRegression(verbose=1, solver='liblinear',random_state=0, C=2, penalty='l2',max_iter=1000)
     ]

models = zip(names, Classifiers)
for name, model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(name)
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print(accuracy_score(y_test, y_pred))


K Nearest Neighbors
[[1758   67]
 [ 959  958]]
              precision    recall  f1-score   support

           0       0.65      0.96      0.77      1825
           1       0.93      0.50      0.65      1917

    accuracy                           0.73      3742
   macro avg       0.79      0.73      0.71      3742
weighted avg       0.79      0.73      0.71      3742

0.7258150721539284
Decision Tree
[[1270  555]
 [ 535 1382]]
              precision    recall  f1-score   support

           0       0.70      0.70      0.70      1825
           1       0.71      0.72      0.72      1917

    accuracy                           0.71      3742
   macro avg       0.71      0.71      0.71      3742
weighted avg       0.71      0.71      0.71      3742

0.7087119187600214
Random Forest
[[1511  314]
 [ 450 1467]]
              precision    recall  f1-score   support

           0       0.77      0.83      0.80      1825
           1       0.82      0.77      0.79      1917

    accuracy   